# Instalacion de dependencias

In [1]:
#pip install happytransformer

In [48]:
import pandas as pd
import numpy as np
import torch
from torch import nn, optim #Pytorch
from datasets import load_dataset

In [49]:
RANDOM_SEED = 42
BATCH_SIZE = 16
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


## Cargar datos

In [50]:
dataset = load_dataset('csv', data_files=['C:/Users/anabe/Desktop/PRUEBAS/APPSTORE1 - ACTUALIZADO/corpus/book4.csv'], split='train')
#C:\Users\anabe\Desktop\PRUEBAS\APPSTORE1 - ACTUALIZADO
datasets = dataset.train_test_split(test_size=0.20) # Test del 20%

Found cached dataset csv (C:/Users/anabe/.cache/huggingface/datasets/csv/default-093a8e447c963488/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)
Loading cached split indices for dataset at C:\Users\anabe\.cache\huggingface\datasets\csv\default-093a8e447c963488\0.0.0\6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317\cache-d995bbfd5f578a3e.arrow and C:\Users\anabe\.cache\huggingface\datasets\csv\default-093a8e447c963488\0.0.0\6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317\cache-974f951674d0b4c2.arrow


In [51]:
datasets

DatasetDict({
    train: Dataset({
        features: ['name', 'general', 'descrip'],
        num_rows: 2496
    })
    test: Dataset({
        features: ['name', 'general', 'descrip'],
        num_rows: 625
    })
})

In [52]:
def etiquetas(examples):
    if examples ["general"] == 'applied science':
        label = 0
    elif examples ["general"] == 'natural sciences':
        label = 1
    elif examples ["general"] == 'math':
        label = 2
    elif examples ["general"] == 'social sciences ':
        label = 3
    elif examples ["general"] == 'art':
        label = 4
    elif examples ["general"] == 'linguistics':
        label = 5
    else:
        label = 6
    return {"labels": label}

In [53]:
dataset = datasets.map(etiquetas)

Loading cached processed dataset at C:\Users\anabe\.cache\huggingface\datasets\csv\default-093a8e447c963488\0.0.0\6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317\cache-8805b999bb461f46.arrow
Loading cached processed dataset at C:\Users\anabe\.cache\huggingface\datasets\csv\default-093a8e447c963488\0.0.0\6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317\cache-0d6d123db84ac67f.arrow


In [54]:
dataset

DatasetDict({
    train: Dataset({
        features: ['name', 'general', 'descrip', 'labels'],
        num_rows: 2496
    })
    test: Dataset({
        features: ['name', 'general', 'descrip', 'labels'],
        num_rows: 625
    })
})

In [55]:
dataset['train'][5]

{'name': 'Ananas Language Exchange',
 'general': 'linguistics',
 'descrip': 'The language app for today. Learn and exchange languages. Think global, be local.',
 'labels': 5}

## Tokenizacion DataSet

In [56]:
model_checkpoint = "roberta-base"

In [57]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
#, use_fast=False

In [58]:
#Vamos a pasar el tokenizador en cada fila del dataset
def tokenize_reviews(examples):
    return tokenizer(examples["descrip"], padding=True, truncation=True)

In [59]:
type(dataset['train']['descrip'][0])

str

In [60]:
encoded_dataset = dataset.map(tokenize_reviews, batched=True )

Map:   0%|          | 0/2496 [00:00<?, ? examples/s]

Map:   0%|          | 0/625 [00:00<?, ? examples/s]

In [61]:
tokenizer.vocab_size

50265

## Cargar el modelo preentrenado

In [62]:
from transformers import AutoModelForSequenceClassification

num_labels = 7
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels).to(device)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.bias', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classi

In [63]:
encoded_dataset['train'].features
#encoded_dataset

{'name': Value(dtype='string', id=None),
 'general': Value(dtype='string', id=None),
 'descrip': Value(dtype='string', id=None),
 'labels': Value(dtype='int64', id=None),
 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}

In [64]:
encoded_dataset.set_format("torch", columns=['input_ids','attention_mask','labels'])
encoded_dataset['train'].features

{'name': Value(dtype='string', id=None),
 'general': Value(dtype='string', id=None),
 'descrip': Value(dtype='string', id=None),
 'labels': Value(dtype='int64', id=None),
 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}

In [65]:
encoded_dataset

DatasetDict({
    train: Dataset({
        features: ['name', 'general', 'descrip', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 2496
    })
    test: Dataset({
        features: ['name', 'general', 'descrip', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 625
    })
})

In [66]:
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(eval_pred):
    labels = eval_pred.label_ids
    preds = eval_pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1}

In [67]:
from transformers import TrainingArguments

model_name = model_checkpoint.split("/")[-1]

batch_size = 64
num_train_epochs=4
num_train_samples = 2_000
train_dataset = encoded_dataset["train"].shuffle(seed=42).select(range(num_train_samples))
#train_dataset = encoded_dataset["train"]

#logging_steps = len(encoded_dataset['train'])//batch_size
logging_steps = len(train_dataset) // (2 * batch_size * num_train_epochs)

training_args = TrainingArguments(
    output_dir="roberta-normal",
    num_train_epochs=num_train_epochs,     
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch", 
    logging_steps=logging_steps,
    #push_to_hub=True,
    #push_to_hub_model_id=f"{model_name}-finetuned-wiki_multi"
)

In [68]:
from transformers import Trainer

trainer = Trainer(
    model=model, 
    args=training_args, 
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=encoded_dataset["test"],
    tokenizer=tokenizer
)

In [69]:
trainer.train()

C:\Users\anabe\anaconda3\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,1.128100,1.021903,0.624000,0.534822
2,0.644900,0.657640,0.769600,0.720253
3,0.577100,0.609698,0.814400,0.798048
4,0.498800,0.578273,0.812800,0.796123


TrainOutput(global_step=128, training_loss=0.8737834971398115, metrics={'train_runtime': 14594.606, 'train_samples_per_second': 0.548, 'train_steps_per_second': 0.009, 'total_flos': 1200498081600000.0, 'train_loss': 0.8737834971398115, 'epoch': 4.0})